### INPUT

In [1]:
# positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
#                       [430, 100], [520, 100], [520, 150],[600, 150], [600, 200], [520, 200]
#                       , [520, 440], [650, 440], [650, 630],
#                       [210, 630], [210, 530], [70, 530], [70, 420],
#                         [150, 420], [150, 140], [50, 140], [50, 40]]
# # [ngang, dọc]
# # grid_size = (60, 100)
# positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
#                       [430, 100], [520, 100], [520, 150],[600, 150], [600, 220], [620, 220],
#                       [620, 180], [660, 180], [660, 280], 
#                       [550,280], [550,220], [520, 220], 
#                       [520, 440], [650, 440], [650, 630],
#                       [210, 630], [210, 530], [70, 530], [70, 420], 
#                       [150, 420], [150, 140], [50, 140], [50, 40]]
positions = [[60, 60], [270, 60], [270, 270], [60, 270], [60, 60]]
grid_size = [50, 100]
# positions = [[50, 40], [150, 40], [150, 120], [350, 120], 
#              [350, 60], [400, 60], [400, 200], [500, 200], 
#              [500, 500], [50, 500], [50, 40]]

### CREATE BORDER LINE

In [2]:
import cv2
import numpy as np
from model.Models import BorderPoint
from model.Models import Matrix
from model.Models import State
from model.Models import Point
from model.Models import Core

In [3]:
border_point = BorderPoint()
border_line = border_point.create_line_border_all(positions)


In [4]:

trung_diem = [int(np.mean(np.array(border_line)[:,0])), int(np.mean(np.array(border_line)[:,1]))]
border1, border2, border3, border4 = border_point.create_line_border(trung_diem)

#### CHECK BORDER LINE

In [5]:

image = np.ones((700, 1100, 3), np.uint8) * 255

[cv2.line(image, (x,y), (x,y), (0,0,255), 3) for [x,y] in border1]
[cv2.line(image, (x,y), (x,y), (200,200,200), 3) for [x,y] in border2]
[cv2.line(image, (x,y), (x,y), (255,0,255), 3) for [x,y] in border3]
[cv2.line(image, (x,y), (x,y), (100,255,255), 3) for [x,y] in border4]
print()

## CREATE MATRIX (1->Cạnh, 0->thịt, 2 -> Góc, None->Vùng ngoài)

In [6]:
trung_diem = [int(np.mean(np.array(border_line)[:,0])), int(np.mean(np.array(border_line)[:,1]))]
border1, border2, border3, border4 = border_point.create_line_border(trung_diem)

mt = Matrix()
#  Tạo ma trận gồm 0, 1 và None
matrix1, matrix2, matrix3, matrix4 = mt.create_matrix(trung_diem, grid_size, border_line, border1, border2, border3, border4)

#  Tìm các đỉnh
corners1 = mt.take_all_corners(matrix1)
corners2 = mt.take_all_corners(matrix2)
corners3 = mt.take_all_corners(matrix3)
corners4 = mt.take_all_corners(matrix4)

#  Thay đổi các dỉnh có giá trị 2 
matrix1 = mt.add_corners_to_matrix(corners1, matrix1)
matrix2 = mt.add_corners_to_matrix(corners2, matrix2)
matrix3 = mt.add_corners_to_matrix(corners3, matrix3)
matrix4 = mt.add_corners_to_matrix(corners4, matrix4)

#  Đánh nhóm cho phần thịt của ma trận
matrix1 = mt.label_group_for_0(matrix1)
matrix2 = mt.label_group_for_0(matrix2)
matrix3 = mt.label_group_for_0(matrix3)
matrix4 = mt.label_group_for_0(matrix4)

#  Gom nhóm các góc
group1 = mt.getGroup(matrix1)
group2 = mt.getGroup(matrix2)
group3 = mt.getGroup(matrix3)
group4 = mt.getGroup(matrix4)

1
2
3
4


In [7]:
mmx = [max(np.array(border_line)[:, 0]) - min(np.array(border_line)[:, 0])]
mmy = [max(np.array(border_line)[:, 1]) - min(np.array(border_line)[:, 1])]
print('x',mmx, 'y',mmy)

x [210] y [210]


In [8]:
arr_top =  np.array(matrix1 + matrix2)
arr_bottom = np.array(matrix3 + matrix4)
matrix = np.concatenate((arr_top, arr_bottom), axis=1)
matrix = mt.label_group_for_0(matrix)
matrix.shape

(211, 211, 4)

In [9]:

image = np.ones((700, 700, 3), np.uint8) * 255
for i in range(0, matrix.shape[0]):
    for j in range(0, matrix.shape[1]):
        cell = matrix[i,j]
        cell_top = matrix[i-1,j]
        x,y,val,gr = cell
        x_,y_,val_,gr_ = cell_top
        if cell[2] == 1:
            cv2.line(image, (x,y), (x,y), (0,0,0),1)
            # cv2.line(image, (x_,y_), (x_,y_), (0,0,255),5)
        if val == 0 and (gr <= 25):
            cv2.line(image, (x,y), (x,y), (10,230,200),1)
        if val is None:
            cv2.line(image, (x,y), (x,y), (200,230,200),1)
            
while True:
    cv2.imshow('Brick', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()

In [10]:
st = State(group1, group2, group3, group4)

pieces_state1 = st.sort_state(st.state1)
pieces_state2 = st.sort_state(st.state2)
pieces_state3 = st.sort_state(st.state3)
pieces_state4 = st.sort_state(st.state4)


# GROUPING CORNERS OF 4 STATE

In [11]:
# check
pieces_state_all = [[[x,y,v,g] for [x,y,v,g] in arr] for arr in pieces_state1 + pieces_state2 + pieces_state3 + pieces_state4] 
corners = []
for arr in range(0, len(pieces_state_all)):
    print(arr+1, pieces_state_all[arr])
    for [x,y,v,g] in pieces_state_all[arr]:
        corners.append([x,y])

unique_pieces = []
seen_coordinates = set()

for x, y in corners:
    if (x, y) not in seen_coordinates:
        unique_pieces.append([x, y])
        seen_coordinates.add((x, y))
len(unique_pieces)

1 [[165, 165, 2, 1], [165, 65, 2, 1], [115, 65, 2, 1], [115, 165, 2, 1], [165, 165, 2, 1]]
2 [[115, 165, 2, 2], [115, 65, 2, 2], [65, 65, 2, 2], [65, 165, 2, 2], [115, 165, 2, 2]]
3 [[65, 165, 2, 3], [65, 65, 2, 3], [60, 65, 2, 3], [60, 165, 2, 3], [65, 165, 2, 3]]
4 [[165, 65, 2, 4], [165, 60, 2, 4], [115, 60, 2, 4], [115, 65, 2, 4], [165, 65, 2, 4]]
5 [[115, 65, 2, 5], [115, 60, 2, 5], [65, 60, 2, 5], [65, 65, 2, 5], [115, 65, 2, 5]]
6 [[65, 65, 2, 6], [65, 60, 2, 6], [60, 60, 2, 6], [60, 65, 2, 6], [65, 65, 2, 6]]
7 [[270, 165, 2, 1], [270, 65, 2, 1], [215, 65, 2, 1], [215, 165, 2, 1], [270, 165, 2, 1]]
8 [[215, 165, 2, 2], [215, 65, 2, 2], [265, 65, 2, 2], [265, 165, 2, 2], [215, 165, 2, 2]]
9 [[265, 165, 2, 3], [265, 65, 2, 3], [270, 65, 2, 3], [270, 165, 2, 3], [265, 165, 2, 3]]
10 [[270, 65, 2, 4], [270, 60, 2, 4], [215, 60, 2, 4], [215, 65, 2, 4], [270, 65, 2, 4]]
11 [[215, 65, 2, 5], [215, 60, 2, 5], [265, 60, 2, 5], [265, 65, 2, 5], [215, 65, 2, 5]]
12 [[265, 65, 2, 6], [265,

35

In [12]:

image = np.ones((700, 700, 3), np.uint8) * 255
for i in range(0, matrix.shape[0]):
    for j in range(0, matrix.shape[1]):
        cell = matrix[i,j]
        cell_top = matrix[i-1,j]
        x,y,val,gr = cell
        x_,y_,val_,gr_ = cell_top
        if cell[2] == 2:
            cv2.line(image, (x,y), (x,y), (0,0,0),5)
            cv2.line(image, (x_,y_), (x_,y_), (0,0,255),5)

while True:
    cv2.imshow('Brick', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()

### CHECK MATRIX

In [13]:
image = np.ones((700, 700, 3), np.uint8) * 255

arr_top =  np.array(matrix1 + matrix2)
arr_bottom = np.array(matrix3 + matrix4)
matrix = np.concatenate((arr_top, arr_bottom), axis=1)
matrix = mt.label_group_for_0(matrix)

for i in range(0, matrix.shape[0]):
    for j in range(0, matrix.shape[1]):
        x,y,val,gr = matrix[i,j]
        if val is None: # in vung ngoai
            cv2.line(image, (x,y), (x,y), (200,200,200),1)
        if val == 0 and (5 <= gr <= 25):
            cv2.line(image, (x,y), (x,y), (10,230,200),1)
            
for i in range(0, matrix.shape[0]):
    for j in range(0, matrix.shape[1]):
        x,y,val,gr = matrix[i,j]
        if val == 1:
            cv2.line(image, (x,y), (x,y), (0,0,0),1)
        if val == 2:
            cv2.line(image, (x,y), (x,y), (0,0,0),5)
            
while True:
    cv2.imshow('Brick', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()

### THỬ NGHIỆM TỪ ĐÂY

In [14]:
cores = []
for x,y in unique_pieces:
    cores.append(Core(point=[x,y]))

[(core.values(), core.getcore(core.point)) for core in cores]

[([[165, 165], False, False, False, False], 0),
 ([[165, 65], False, False, False, False], 0),
 ([[115, 65], False, False, False, False], 0),
 ([[115, 165], False, False, False, False], 0),
 ([[65, 65], False, False, False, False], 0),
 ([[65, 165], False, False, False, False], 0),
 ([[60, 65], False, False, False, False], 0),
 ([[60, 165], False, False, False, False], 0),
 ([[165, 60], False, False, False, False], 0),
 ([[115, 60], False, False, False, False], 0),
 ([[65, 60], False, False, False, False], 0),
 ([[60, 60], False, False, False, False], 0),
 ([[270, 165], False, False, False, False], 0),
 ([[270, 65], False, False, False, False], 0),
 ([[215, 65], False, False, False, False], 0),
 ([[215, 165], False, False, False, False], 0),
 ([[265, 65], False, False, False, False], 0),
 ([[265, 165], False, False, False, False], 0),
 ([[270, 60], False, False, False, False], 0),
 ([[215, 60], False, False, False, False], 0),
 ([[265, 60], False, False, False, False], 0),
 ([[270, 270

### `TRỰC QUAN HÓA KẾT QUẢ TỔNG KHUNG HÌNH`

In [15]:
# import numpy as np
# import cv2
# 
# image = np.ones((700, 700, 3), np.uint8) * 255
# shape_res = np.array(matrix).shape
# 
# for i in range(0, shape_res[0]):
#     for j in range(0, shape_res[1]):
#         cell = matrix[i][j]
#         x,y,val,gr = cell
#         if cell[2] == 1: # in cạnh
#             cv2.line(image, (x,y), (x,y), (0,0,0),1)
#         elif cell[2] == 2:
#             cv2.line(image, (x,y), (x,y), (0,0,0),7)
# 
# 
# [cv2.line(image, (x,y), (x,y), (255,0,0), 5) for [x,y] in core90]
# [cv2.line(image, (x,y), (x,y), (0,255,0), 5) for [x,y] in core180]
# [cv2.line(image, (x,y), (x,y), (0,0,245), 5) for [x,y] in core360]
# 
# while True:
#     cv2.imshow('Brick', image)
#     if cv2.waitKey(20) & 0xFF == 27:
#         break
# 
# cv2.destroyAllWindows()

In [16]:
# import numpy as np
# import cv2

# image = np.ones((700, 700, 3), np.uint8) * 255
# shape_res = np.array(matrix2).shape

# shape = np.shape(result1)

# for i in range(0, shape[0]):
#     for j in range(0, shape[1]):
#         [x, y, v, g] = result1[i][j]
#         if g != None:
#             cv2.line(image, (x,y), (x,y), (0,0,0),1)
            

# # [cv2.line(image, (x,y), (x,y), (255,0,0), 5) for [x,y] in core90]
# # [cv2.line(image, (x,y), (x,y), (0,255,0), 5) for [x,y] in core180]
# # [cv2.line(image, (x,y), (x,y), (0,0,245), 5) for [x,y] in core360]

# while True:
#     cv2.imshow('Brick', image)
#     if cv2.waitKey(20) & 0xFF == 27:
#         break

# cv2.destroyAllWindows()

In [17]:
# def minmax_XY(cells):
#     min_max_arrX = [min(np.array(cells)[:,0]), max(np.array(cells)[:, 0])]
#     min_max_arrY = [min(np.array(cells)[:,1]), max(np.array(cells)[:, 1])]
#     return min_max_arrX, min_max_arrY
# 
# def trong_tam(arr):
#     return [int(np.mean(np.array(arr)[:,0])), int(np.mean(np.array(arr)[:,1]))]
# 
# def khoang_cach(X, Y):
#     return math.sqrt((Y[0] - X[0]) ** 2 + (Y[1] - X[1]) ** 2)
# 
# ind = 1
# 
# import cv2
# import math
# 
# image = np.ones((700, 700, 3), np.uint8) * 255
# 
# # tính khoảng cách ngắn nhất của 2 điểm tọa độ
# 
# for index in range(0, len(result)):
#     if ind >= 1:
#         
#         item = [[x,y,val,gr] for [x,y,val,gr] in result[index]]
#         cv2.line(image, (item[0][0], item[0][1]), (item[0][0], item[0][1]), (0,0,100), 5)
#         print('yet sort:', result[index])
#         for i in range(0, len(result[index])-1):
#             check_kc = khoang_cach([result[index][i][0], result[index][i][1]], [result[index][i+1][0], result[index][i+1][1]])
#             # print(check_kc, [result[index][i][0], result[index][i][1]], [result[index][i+1][0], result[index][i+1][1]])
#             check_index = i
#             item_checking = result[index][i]
#             
#             for j in range(i + 1, len(result[index])):
#                 kc = khoang_cach([result[index][i][0], result[index][i][1]], [result[index][j][0], result[index][j][1]])
#                 # print(result[index][i], result[index][j], kc, end='')
#                 # print([item[j][0], item[j][1]], kc)
#                 if kc >= check_kc:
#                     check_kc = kc
#                     # print('\t\tloc_min', result[index][j], end='')
#                     check_index = j
#                 # print()
#                     
#             result[index][i], result[index][check_index] = result[index][check_index], result[index][i]
#             # print()
#         print('sorted:',result[index])
#         
#         for ii in range(0, len(result[index])-1):
#             loc = (result[index][ii][0], result[index][ii][1])
#             loc_next = (result[index][ii + 1][0], result[index][ii + 1][1])
#             cv2.line(image, loc, loc_next, (0,255,0), 2)
#         # cv2.line(image, loc_next, (result[index][0][0], result[index][0][1]), (0,255,0), 2)
#     ind += 1
#         
# # [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in cell_edges]
# # [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in canh1]
# # [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in canh2]
# # [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in canh3]
# # [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in canh4]
# 
# while True:
#     cv2.imshow('DatGach', image)
#     if cv2.waitKey(20) & 0xFF == 27:
#         break
# 
# cv2.destroyAllWindows()

In [18]:
from itertools import groupby
import random as rd
image = np.ones((700, 700, 3), np.uint8) * 255
# Lọc lây toàn bộ thịt
array_thit = [[x,y,v,g] for x,y,v,g in np.array(matrix2).reshape(-1,4).tolist() if g != None]
# Sắp xếp theo group
r_1 = sorted(array_thit, key= lambda x: x[3])
# gom thịt có group giống nhau thành 1 danh sách con
groups_thit = [list(group) for key, group in groupby(r_1, key=lambda x: x[3])]
# Sắp xếp mảng thịt có số lượng từ lớn đến bé
groups_thit = sorted(groups_thit, key=len, reverse=True)
print(len(groups_thit))
check = len(groups_thit[0])
color = (255,0,0)
for arr in groups_thit:
    
    if check - len(arr) != 0:
        color = (rd.randint(0,250), rd.randint(0,250), rd.randint(200,255))
        check = len(arr)
        
    for [x,y,val,gr] in arr:
        if val == 0:
            cv2.line(image, (x,y), (x,y), color,1)

# [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in cell_edges]
# [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in canh1]
# [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in canh2]
# [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in canh3]
# [cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in canh4]

while True:
    cv2.imshow('DatGach', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()

6
